# Import packages and preliminary operations

In [29]:
# Installazione pacchetti
# %pip install nltk
# %pip install gensim
# %pip install simplemma
# %pip install spacy

In [1]:
import string
import sys
import io 
import nltk # Natural Language Processing package

# Stopwords
from gensim.parsing.preprocessing import remove_stopwords
from nltk.corpus import stopwords 
nltk.download('stopwords')
stops = set(stopwords.words('italian'))
nltk_stopwords = nltk.corpus.stopwords.words('italian')
# Valutare di aggiungere eventuali forme arcariche delle stopwords italiane, o se c'è necessità di estendere questa lista

# Bigrammi
from gensim.models import Phrases  

# Per dividere il testo in frasi in base ai punti
nltk.download('punkt')

# Per tokenizzare
from nltk.tokenize import word_tokenize 

# Lemmatization
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

# Position tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

import simplemma # altro pacchetto per lemmatizzare

import spacy # altro pacchetto per lemmatizzare
import it_core_news_lg

# Contatore parole uniche
from collections import Counter

# Per esplorare risultati
import random

import unicodedata # Pulizia dei caratteri speciali

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# Lista stopwords ulteriore (no forme arcaiche presenti)
import requests
url = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-it/master/stopwords-it.txt'
resp = requests.get(url)

with open("stopwords_ext.txt", 'w') as file:
    file.write(resp.text)

In [3]:
def strip_accents(text): # Normalizzo tutte le parole togliendo caratteri speciali tipo accenti
    try:
        text = unicode(text, 'utf-8')
    except NameError:
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return text

def append_new_line(file_name, text_to_append):

    # Apro il file in 'append' e 'lettura'
    with open(file_name, "a+") as file:
        file.seek(0)

        # Se il file non è vuoto vado a capo
        if len(file.read(100)) > 0:
            file.write("\n")
        # Inserisco la stringa
        file.write(text_to_append)
with open ('stopwords_ext.txt', 'r') as f:
    for line in f:
        append_new_line('Output/stopwords_ext_clean.txt', strip_accents(line))

In [4]:
# Estensione della lista delle stopwords (al momento non sono presenti forme arcaiche)
stopwords_ext = []
with open('Output/stopwords_ext_clean.txt') as f:
    for line in f:
         stopwords_ext.append(line)
stopwords_ext = [s.replace('\n', '') for s in stopwords_ext]

# Unisco la lista di stopwords di nltk con quella estratta dal txt
stopwords = list(nltk_stopwords)
stopwords.extend(x for x in stopwords_ext if x not in stopwords)

# Bigrammi

In [ ]:
from gensim.models.phrases import Phrases, Phraser
import string
import os
from os import listdir

In [5]:
def build_phrases(sentences):
    phrases = Phrases(sentences,
                      min_count=5,
                      threshold=10,
                      progress_per=1000)
    return Phraser(phrases)

In [6]:
# Questa funzione ricostruisce la frase con i bigrammi
def sentence_to_bi_grams(phrases_model, sentence):
    return ' '.join(phrases_model[sentence])

In [ ]:
def simple_preproc(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [15]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/autori/')

# Unisco tutti i corpus degli autori in un unico corpus
path = 'C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/autori/'
filenames =  listdir(path)

with open(path + 'total_corpus.txt', 'w', encoding='utf-8') as outfile:
    for fname in filenames:
        with open(fname, encoding='utf-8') as infile:
            outfile.write(infile.read())

# Eseguo il simple pre-processing sull'intero corpus
with open('full_corpus.txt', encoding='utf-8') as read_file:
    sentences = [simple_preproc(k).lower().split() for k in read_file.readlines()]
full_corpus_models = build_phrases(sentences)
full_corpus_models.save('bigrams_full.txt')

In [121]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Preprocessing/periodi_storici')
    
with open('tardo_medioevo_sentences_nl.txt', encoding='utf-8') as read_file:
    sentences = [simple_preproc(k).lower().split() for k in read_file.readlines()]
phrases_model = build_phrases(sentences)
phrases_model.save('phrases_model_tardo_medioevo_nl.txt')

# Pre-processing functions

## Pre-processing considerando i bigrammi

In [124]:
# Adatto la funzione di preprocessing per inserire anche i bigrammi

def preprocessing_w_bigrams(file_name):

  output=""
  with open(file_name, encoding='utf-8') as f:
      for line in f:
          if not line.isspace(): # Rimuovo linee vuote
              output+=line

  # Divido il testo in frasi, basandomi sui punti
  output_sentences = nltk.tokenize.sent_tokenize(output)

  # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze ecc..
  # output_sentences = output_sentences[:-]

  filtered_sentences = []
  # 'Pulisco' ogni frasi, una alla volta
  for sentence in output_sentences:
    # Metto tutto in lower case
    lower_sentence=sentence.lower()
    # Rimuovo caratteri non alfa numerici
    noalfa_sentence = [w for w in word_tokenize(lower_sentence) if (w.isalpha()==True)]
    # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
    filtered_sentence = [w for w in noalfa_sentence if ((w not in stops) and (len(w) > 1))]
    # Valuto i bigrammi
    bigram_sentence = sentence_to_bi_grams(phrases_model, filtered_sentence)
    bigram_sentence = bigram_sentence.split() # Rimetto tutto in token
    # Ricostruisco la lista con le frasi 'pulite'
    if bigram_sentence:
      filtered_sentences.append(bigram_sentence)

  return filtered_sentences

## Pre-processing senza lemmittazione

In [4]:
def preprocessing(file_name):

  output=""
  with open(file_name, encoding='utf-8') as f:
      for line in f:
          if not line.isspace(): # Rimuovo linee vuote
              output+=line

  # Divido il testo in frasi, basandomi sui punti
  output_sentences = nltk.tokenize.sent_tokenize(output)

  # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze,...
  # output_sentences = output_sentences[:-]

  filtered_sentences = []
  # 'Pulisco' ogni frase, una alla volta
  for sentence in output_sentences:
    # Metto tutto in lower case
    lower_sentence=sentence.lower()
    # Rimuovo caratteri non alfa numerici
    noalfa_sentence = [w for w in word_tokenize(lower_sentence) if (w.isalpha()==True)]
    # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
    filtered_sentence = [w for w in noalfa_sentence if ((w not in stops) and (len(w) > 1))]
    # Ricostruisco la lista con le frasi 'pulite'
    if filtered_sentence:
      filtered_sentences.append(filtered_sentence)

  return filtered_sentences

## Pre-processing con lemmatizzazione

In [40]:
def preprocessing_lem(file_name):

    output = ""
    with open(file_name, encoding='utf-8') as f:
        for line in f:
            if not line.isspace():  # Rimuovo linee vuote
                output += line

    # Divido il testo in frasi, basandomi sui punti
    output_sentences = nltk.tokenize.sent_tokenize(output)

    # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze, ...
    # output_sentences = output_sentences[:-]

    filtered_sentences = []
    # 'Pulisco' ogni frasi, una alla volta
    for sentence in output_sentences:
        # Metto tutto in lower case
        lower_sentence = sentence.lower()
        # Rimuovo caratteri non alfa numerici
        noalfa_sentence = [w for w in word_tokenize(
            lower_sentence) if (w.isalpha() == True)]
        # Lemmatize
        lemmatized_sentence = [lemmatizer.lemmatize(
            w, get_wordnet_pos(w)) for w in noalfa_sentence]
        # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
        filtered_sentence = [w for w in lemmatized_sentence if (
            (w not in stops) and (len(w) > 1))]
        # Ricostruisco la lista con le frasi 'pulite'
        if filtered_sentence:
            filtered_sentences.append(filtered_sentence)

    return filtered_sentences


## Pre-processing con lemmatizzazione w/Simplemma

In [90]:
import simplemma

# Preprocessing (CON lemmatization w/ SIMPLEMMA)


def preprocessing_simplemma(file_name):

    output = ""
    with open(file_name, encoding='utf-8') as f:
        for line in f:
            if not line.isspace():  # Rimuovo linee vuote
                output += line

    # Divido il testo in frasi, basandomi sui punti
    output_sentences = nltk.tokenize.sent_tokenize(output)

    # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze, ...
    # output_sentences = output_sentences[:-]

    filtered_sentences = []
    # 'Pulisco' ogni frasi, una alla volta
    for sentence in output_sentences:
        # Metto tutto in lower case
        lower_sentence = sentence.lower()
        # Rimuovo caratteri non alfa numerici
        noalfa_sentence = [w for w in word_tokenize(
            lower_sentence) if (w.isalpha() == True)]
        # Lemmatize
        lemmatized_sentence = [simplemma.lemmatize(
            w, lang='it') for w in noalfa_sentence]
        # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
        filtered_sentence = [w for w in lemmatized_sentence if (
            (w not in stops) and (len(w) > 1))]
        # Ricostruisco la lista con le frasi 'pulite'
        if filtered_sentence:
            filtered_sentences.append(filtered_sentence)

    return filtered_sentences


## Pre-processing con lemmatizzazione w/Spacy

In [42]:
import spacy
import it_core_news_lg

nlp = it_core_news_lg.load()

# Preprocessing (CON lemmatization w/ SPACY). In questo caso ho rimosso le stopwords prima della lemmatizzazione, va provato
# con tutti a fare così

def preprocessing_spacy(file_name):

  output=""
  with open(file_name, encoding='utf-8') as f:
      for line in f:
          if not line.isspace(): # Rimuovo linee vuote
              output+=line

  # Divido il testo in frasi, basandomi sui punti
  output_sentences = nltk.tokenize.sent_tokenize(output)

  # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze, ...
  # output_sentences = output_sentences[:-]

  filtered_sentences = []
  # 'Pulisco' ogni frasi, una alla volta
  for sentence in output_sentences:
    # Metto tutto in lower case
    lower_sentence=sentence.lower()
    # Rimuovo caratteri non alfa numerici
    noalfa_sentence = [w for w in word_tokenize(lower_sentence) if (w.isalpha()==True)]
    # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
    nostop_sentence = [w for w in noalfa_sentence if ((w not in stops) and (len(w) > 1))]
    # Lemmatize
    lemmatized_sentence = " ".join([w for w in nostop_sentence])
    doc = nlp(lemmatized_sentence)
    filtered_sentence = [w.lemma_ for w in doc]
    # Ricostruisco la lista con le frasi 'pulite'
    if filtered_sentence:
      filtered_sentences.append(filtered_sentence)

  return filtered_sentences

# Valutazione delle varie funzioni di pre-processing

In [2]:
# Funzione per esplorare i risultati
def checkresults(sentences):
  words = []
  for i in range(len(sentences)):
    for j in range(len(sentences[i])):
      words.append(sentences[i][j])

  # Numero totale di parole
  print('Parole: ' + str(len(words)))

  # Numero di parole uniche
  word_count = Counter(words)
  keys = word_count.keys()
  print('Parole uniche: '+ str(len(keys)))

  # Parole più comuni
  print('Parole più comuni:')
  print(word_count.most_common(10))

  # Stampo alcune frasi random
  print('Frasi random:')
  randomlist = random.sample(range(0, len(sentences)), 5)
  for i in randomlist:
    print(sentences[i])

In [5]:
def savesentences(filename, sentences):
  os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Preprocessing')
  with open(filename + '.txt', 'w', encoding='utf-8') as fp:
    for sentence in sentences:
      fp.write(str(sentence) + '\n')

In [6]:
# Senza lemmatizzazione
def savesentencesnl(filename, sentences):
  os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Preprocessing')
  with open(filename + '_nl.txt', 'w', encoding='utf-8') as fp:
    for sentence in sentences:
      fp.write(str(sentence) + '\n')

# Pre-processing

In [7]:
from os import listdir

directory = 'C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/periodi_storici'
files_dir =  listdir(directory)
periodi_storici_files = []
for names in files_dir:
    if names.endswith(".txt"):
        periodi_storici_files.append(names)
print (periodi_storici_files)

['1000_1250_Pieno Medievo.txt', '1250_1454_Tardo_Medioevo.txt', '1454_1559_Rinascimento.txt', '1559_1707_Seicento.txt', '1707_1796_Settecento.txt', '1796_1814_Periodo_napoleonico.txt', '1814_1860_Risorgimento.txt', "1861_1914_L'Italia_liberale.txt", "1915_1922_L'Italia_nella_Prima_Guerra_Mondiale_e_dopoguerra.txt", '1922_1945_Ventennio_Fascista_Seconda_Guerra_Mondiale.txt', '1945_presente_La_Repubblica_Italiana.txt']


## Corpus per periodi storici

### Senza lemmatizzazione

In [69]:
import os

#### Corpus 'Pieno medioevo'

In [70]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/periodi_storici')
pieno_medioevo_sentences = []
pieno_medioevo_sentences = preprocessing(periodi_storici_files[0])

In [71]:
checkresults(pieno_medioevo_sentences)

Parole: 3732
Parole uniche: 1758
Parole più comuni:
[('no', 51), ('sì', 51), ('de', 48), ('donna', 44), ('core', 38), ('né', 35), ('et', 28), ('ben', 26), ('ca', 25), ('gran', 25)]
Frasi random:
['dunque', 'madonna', 'voglio', 'sofrire', 'far', 'sembianti', 'contrata', 'gente', 'sforza', 'maldire', 'però', 'blasmata', 'diletta', 'dire', 'male', 'bene', 'fiata']
['foco', 'inchiuso', 'poi', 'passa', 'difore', 'lostrore', 'sanza', 'far', 'rotura', 'così', 'ochi', 'pass', 'core', 'no', 'persona', 'figura']
['gran', 'disio', 'dipinsi', 'pintura', 'bella', 'simigliante', 'quando', 'vio', 'guardo', 'figura', 'par', 'davante', 'crede', 'salvarsi', 'fede', 'ancor', 'veggia', 'inante']
['conventi', 'ritenere', 'donaomi', 'gio', 'rimembranza', 'allegramente']
['xxvi', 'claro', 'vista', 'ploggia', 'dare', 'scuro', 'rendere', 'clarore', 'foco', 'arzente', 'ghiaccia', 'diventare', 'freda', 'neve', 'rendere', 'calore', 'dolze', 'cose', 'molto', 'amareare', 'de', 'rendere', 'dolzore', 'dui', 'guerreri

In [60]:
savesentencesnl('pieno_medioevo_sentences', pieno_medioevo_sentences)

#### Corpus tardo medioevo

In [49]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/periodi_storici')
tardo_medioevo_sentences = preprocessing(periodi_storici_files[1])

In [55]:
checkresults(tardo_medioevo_sentences)

Parole: 2622917
Parole uniche: 139172
Parole più comuni:
[('de', 27661), ('sì', 16223), ('poi', 11661), ('me', 11227), ('così', 10646), ('ciò', 9992), ('quando', 9814), ('disse', 9420), ('cosa', 9365), ('cioè', 8821)]
Frasi random:
['gridai', 'alto', 'dio', 'signore', 'render', 'posso', 'tanti', 'benefici', 'ami', 'te', 'core']
['però', 'dice', 'ten', 'spalle', 'volte', 'verso', 'damiata', 'aciò', 'mondo', 'quel', 'aver', 'respeto', 'parte', 'mondo']
['el', 'sire', 'iddio', 'potenza', 'immergerà', 'tutta', 'possa', 'scura', 'fossa', 'lì', 'sempre', 'viso', 'lordo', 'già', 'mai', 'de', 'fia', 'recordo']
['cioè', 'tosto', 'pena', 'purgati']
['egli', 'fitto', 'capo', 'ponte', 'colonna', 'marmo', 'sotto', 'colonna', 'hae', 'lione', 'marmo', 'sopra', 'altro', 'molto', 'begli', 'grandi', 'ben', 'fatti', 'lungi', 'colonna', 'passo', 'altra', 'né', 'piú', 'né', 'meno', 'fatta', 'due', 'leoni', 'colonna', 'altra', 'chiuso', 'tavole', 'marmo', 'percioché', 'niuno', 'potesse', 'cadere', 'acqua']


In [63]:
savesentencesnl('tardo_medioevo_sentences', tardo_medioevo_sentences)

#### Corpus Rinascimento

In [73]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/periodi_storici')
rinascimento_sentences = preprocessing(periodi_storici_files[2])

In [76]:
savesentencesnl('rinascimento_sentences', rinascimento_sentences)

#### Corpus Seicento

In [74]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/periodi_storici')
seicento_sentences = preprocessing(periodi_storici_files[3])

In [77]:
savesentencesnl('seicento_sentences', seicento_sentences)

#### Corpus Settecento

In [8]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/periodi_storici')
settecento_sentences = preprocessing(periodi_storici_files[4])

In [9]:
savesentencesnl('settecento_sentences', settecento_sentences)

In [10]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/periodi_storici')
La_repubblica_italiana = preprocessing(periodi_storici_files[10])

In [11]:
savesentencesnl('La_repubblica_italiana', La_repubblica_italiana)

## Corpus per fasi letterarie

### Senza lemmatizzazione (campione)

In [80]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/fasi_letterarie')
avanguardie_primo_novecento_senteces = []
avanguardie_primo_novecento_senteces = preprocessing('11_avanguardie_primo_novecento.txt')

In [81]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Preprocessing/fasi_letterarie')
savesentencesnl('avanguardie_primo_novecento_senteces', avanguardie_primo_novecento_senteces)

In [129]:
checkresults(avanguardie_primo_novecento_senteces)

Parole: 344934
Parole uniche: 50367
Parole più comuni:
[('due', 1297), ('senza', 1216), ('fra', 1150), ('quando', 965), ('ogni', 951), ('così', 950), ('me', 914), ('sempre', 871), ('cosa', 838), ('ancora', 801)]
Frasi random:
['dice', 'mezza', 'voce', 'scatti', 'sangue', 'romagnolo']
['madre', 'teneva', 'gambe', 'mostrandolo', 'sotto', 'meraviglia', 'infatti', 'piccolo', 'agitandosi', 'esponeva', 'carni', 'meravigliose', 'freschezza', 'colore']
['poteva', 'dire', 'esattezza', 'pena', 'fragili', 'membra', 'sembravano', 'lacerate']
['re', 'baldoria']
['silenzio']


### Con lemmatizzazione (campione)

In [82]:
# LEMMATIZZAZIONE W/ NLTK
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/fasi_letterarie')
avanguardie_primo_novecento_senteces_nltk = preprocessing_lem('11_avanguardie_primo_novecento.txt')

In [87]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Preprocessing/fasi_letterarie')
savesentences('avanguardie_primo_novecento_senteces_nltk', avanguardie_primo_novecento_senteces_nltk)

In [100]:
# LEMMATIZZAZIONE W/ SIMPLEMMA
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/fasi_letterarie')
avanguardie_primo_novecento_senteces_simplemma = preprocessing_simplemma('11_avanguardie_primo_novecento.txt')

In [103]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Preprocessing/fasi_letterarie')
savesentences('avanguardie_primo_novecento_senteces_simplemma', avanguardie_primo_novecento_senteces_simplemma)

In [101]:
# LEMMATIZZAZIONE W/ SPACY 
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/fasi_letterarie')
avanguardie_primo_novecento_senteces_spacy = preprocessing_spacy('11_avanguardie_primo_novecento.txt')

In [102]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Preprocessing/fasi_letterarie')
savesentences('avanguardie_primo_novecento_senteces_spacy', avanguardie_primo_novecento_senteces_spacy)

In [104]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/fasi_letterarie')
classicismo_arcadia_sentences_spacy = preprocessing_spacy('6_classicismo_arcadia.txt')

In [105]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Preprocessing/fasi_letterarie')
savesentences('classicismo_arcadia_sentences_spacy', classicismo_arcadia_sentences_spacy)

In [110]:
checkresults(avanguardie_primo_novecento_senteces)

Parole: 344934
Parole uniche: 50367
Parole più comuni:
[('due', 1297), ('senza', 1216), ('fra', 1150), ('quando', 965), ('ogni', 951), ('così', 950), ('me', 914), ('sempre', 871), ('cosa', 838), ('ancora', 801)]
Frasi random:
['dietro', 'minaccia', 'serpenti', 'elefanti']
['rendetemela']
['bevono', 'dimenticando', 'austriaci']
['tre', 'donne', 'avventano', 'signor', 'tenente', 'creda', 'canaglie']
['eccoti', 'qua', 'baldoria']


In [115]:
checkresults(avanguardie_primo_novecento_senteces_nltk)

Parole: 344902
Parole uniche: 50262
Parole più comuni:
[('due', 1297), ('senza', 1216), ('fra', 1150), ('quando', 965), ('ogni', 951), ('così', 950), ('me', 919), ('sempre', 871), ('cosa', 838), ('os', 802)]
Frasi random:
['giovane', 'brutto', 'forte', 'buona', 'troppo', 'grossolano', 'origine']
['funerale']
['donna', 'dà', 'milano', 'reticenze', 'mezzetinte', 'parentesi', 'sospensioni', 'spalancherebbe', 'invece', 'brutalmente', 'generosamente', 'nervi', 'spirito', 'corpo', 'stesso', 'uomo', 'trovasse', 'rom']
['svalutazione', 'pericolosa', 'aleatoria', 'industria', 'forestiero']
['figlio']


In [113]:
checkresults(avanguardie_primo_novecento_senteces_simplemma)

Parole: 360875
Parole uniche: 29090
Parole più comuni:
[('essere', 12083), ('avere', 6158), ('fare', 2793), ('potere', 1755), ('sapere', 1413), ('volere', 1341), ('vedere', 1319), ('due', 1297), ('dire', 1297), ('cosa', 1267)]
Frasi random:
['badare', 'prendere', 'responsabilità', 'ricostruire', 'dialogare', 'testualmente']
['trentatré', 'anno', 'or', 'essere', 'essere', 'messo', 'lassù', 'camino', 'giusto', 'quando', 'nascere']
['sorridere', 'tacendo', 'mentre', 'attraversare', 'persona', 'terrena', 'già', 'trasparire', 'mobile', 'fiorame', 'parete']
['silenziare', 'assoluto']
['finire', 'casa', 'abitare', 'uomo', 'cominciare', 'popolare', 'scimmia']


In [114]:
checkresults(avanguardie_primo_novecento_senteces_spacy)

Parole: 344934
Parole uniche: 38502
Parole più comuni:
[('potere', 1650), ('fare', 1618), ('tutto', 1334), ('due', 1297), ('dire', 1274), ('cosa', 1266), ('vedere', 1259), ('volere', 1246), ('sapere', 1224), ('senza', 1216)]
Frasi random:
['voce', 'femminile', 'discesa', 'parecchio', 'volta', 'venire', 'vedere', 'cancello', 'bene', 'chiuso', 'comandare', 'eunuchio', 'uccidere ti']
['follare', 'infatti', 'disposto', 'assiepatissima', 'tutto', 'cerchio', 'arena']
['quando', 'raggiungere', 'grande', 'ritmo', 'umano', 'condurre', 'stupendo', 'città', 'libertà', 'spirituale']
['neanche']
['due', 'estrema', 'casta', 'dividono', 'suddividono', 'infinito']


### Bigrams without Lemmatization

In [132]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Corpora/fasi_letterarie')
avanguardie_primo_novecento_senteces_bigrams = preprocessing_w_bigrams('11_avanguardie_primo_novecento.txt')

In [10]:
os.chdir('C:/Users/marco/OneDrive - Università degli Studi di Milano-Bicocca/data/Preprocessing/bigrammi/')
checkresults('Alessandro Manzoni.txt')

Parole: 22
Parole uniche: 16
Parole più comuni:
[('n', 3), ('s', 2), ('a', 2), ('o', 2), ('t', 2), ('A', 1), ('l', 1), ('e', 1), ('d', 1), ('r', 1)]
Frasi random:
o
e
o
r
n


In [3]:
checkresults(avanguardie_primo_novecento_senteces)

NameError: name 'avanguardie_primo_novecento_senteces' is not defined

# Topic modelling

Si tratta di una cosa che potrebbe avere senso provare a fare mettendo insieme più libri, quindi costruendo i vari corpus in base all'autore, genere letterario o altro. Non ha senso farlo con le frasi di un singolo libro, in quanto, come si vede sotto, le parole più rilevanti risulterebbero quelle usate per costruire frasi di senso compiuto, come verbi o preposizioni. Invece, usando dei corpus che raccolgono più libri, sulla base delle statistiche della parole presenti in ciascun documento rispetto al totale, il modello riesce a individuare l'equilibrio fra i vari argomenti e valutare quali sono i più rilevanti.

In [ ]:
import gensim
from gensim.utils import simple_preprocess

In [ ]:
decameron_sentences = open('decameron_frasi.txt').read().splitlines()

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True rimuove la punteggiatura
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
decameron_words = list(sent_to_words(decameron_sentences))

In [ ]:
print(decameron_words[:1][0][:30])

In [ ]:
import gensim.corpora as corpora

# Creo un dizionario
id2word = corpora.Dictionary(decameron_words)
# Creo un corpus
texts = decameron_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
from pprint import pprint

# Numero di topics
num_topics = 10
# Definisco il modello LDA
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Stampo le keyword associate ai topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
decameron_sentences[0]